### Imports

In [202]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

### Data

In [203]:
mangaHIall = Table.read('input/mangaHIall_classified_corrected.txt', format = 'ascii.commented_header')
manga = Table.read("input/Pipe3D-master_file_vflag_10_smooth2p27_N2O2_noWords.txt", format="ascii.commented_header")

### Adding vflag values from manga to mangaHIall

In [204]:
vflag0_boolean = mangaHIall['vflag'] == 0.0
vflag1_boolean = mangaHIall['vflag'] == 1.0

vflag_0_or_1_boolean = np.logical_or(vflag0_boolean, vflag1_boolean)

#overwriting mangaHIall table and getting rid of rows with vflag != 0.0 or 1.0
mangaHIall = mangaHIall[vflag_0_or_1_boolean]

### Adding HI values from logHI

In [205]:
mangaHIall['HI'] = np.ones(len(mangaHIall), dtype = np.float64)
mangaHIall['HI'] = 10**(mangaHIall['logHI'])

### Making file with only positive WF50 data

In [206]:
WF50_pos_boolean = mangaHIall['WF50_corrected'] > 0.0
mangaHIall_WF50_positive = mangaHIall[WF50_pos_boolean]

### Voids and walls classification

In [209]:
mangaHIall_WF50_positive['NSA_Mstar' + '-' + 'walls'] = np.NaN * np.ones(len(mangaHIall_WF50_positive), dtype=np.float64)
mangaHIall_WF50_positive['NSA_Mstar' + '-' + 'voids'] = np.NaN * np.ones(len(mangaHIall_WF50_positive), dtype=np.float64)
mangaHIall_WF50_positive['HI' + '-' + 'walls'] = np.NaN * np.ones(len(mangaHIall_WF50_positive), dtype=np.float64)
mangaHIall_WF50_positive['HI' + '-' + 'voids'] = np.NaN * np.ones(len(mangaHIall_WF50_positive), dtype=np.float64)

for i in range(len(mangaHIall_WF50_positive)):
    if mangaHIall_WF50_positive['vflag'][i] == 0.0:
        mangaHIall_WF50_positive['NSA_Mstar' + '-' + 'walls'][i] = mangaHIall_WF50_positive['NSA_Mstar'][i]
        mangaHIall_WF50_positive['HI' + '-' + 'walls'][i] = mangaHIall_WF50_positive['HI_corrected'][i]
    elif mangaHIall_WF50_positive['vflag'][i] == 1:
        mangaHIall_WF50_positive['NSA_Mstar' + '-' + 'voids'][i] = mangaHIall_WF50_positive['NSA_Mstar'][i]
        mangaHIall_WF50_positive['HI' + '-' + 'voids'][i] = mangaHIall_WF50_positive['HI_corrected'][i]

walls_boolean = mangaHIall_WF50_positive['vflag'] == 0.0
voids_boolean = mangaHIall_WF50_positive['vflag'] == 1.0
classified_boolean = np.logical_or(walls_boolean, voids_boolean)
mangaHIall_WF50_positive_classified = mangaHIall_WF50_positive[classified_boolean]

#     walls_boolean = mangaHIall['vflag] == 0.0
#     mangaHIall['walls'][i] = mangaHIall['NSA_Mstar'][i]
#     voids_boolean = mangaHIall['vflag'] == 1.0
#     mangaHIall['voids'][voids_boolean] = mangaHIall['NSA_Mstar'][i]


### Making table with only walls

In [210]:
mangaHIall_Mstar_walls_boolean = np.isfinite(mangaHIall_WF50_positive_classified['NSA_Mstar-walls'])
mangaHIall_HI_walls_boolean = np.isfinite(mangaHIall_WF50_positive_classified['HI-walls'])

mangaHIall_walls_boolean = np.logical_and(mangaHIall_Mstar_walls_boolean, mangaHIall_HI_walls_boolean)
mangaHIall_walls = mangaHIall_WF50_positive_classified[mangaHIall_walls_boolean]
mangaHIall_walls.remove_column('NSA_Mstar-voids')
mangaHIall_walls.remove_column('HI-voids')

### Making table with only voids

In [211]:
mangaHIall_Mstar_voids_boolean = np.isfinite(mangaHIall_WF50_positive_classified['NSA_Mstar-voids'])
mangaHIall_HI_voids_boolean = np.isfinite(mangaHIall_WF50_positive_classified['HI-voids'])

mangaHIall_voids_boolean = np.logical_and(mangaHIall_Mstar_voids_boolean, mangaHIall_HI_voids_boolean)
mangaHIall_voids = mangaHIall_WF50_positive_classified[mangaHIall_voids_boolean]
mangaHIall_voids.remove_column('NSA_Mstar-walls')
mangaHIall_voids.remove_column('HI-walls')

### Writing classified files

In [218]:
mangaHIall_WF50_positive.write('input/mangaHIall_WF50_positive.txt', format = 'ascii.commented_header', overwrite = True)
mangaHIall_voids.write('input/mangaHIall_voids.txt', format = 'ascii.commented_header', overwrite = True)
mangaHIall_walls.write('input/mangaHIall_walls.txt', format = 'ascii.commented_header', overwrite = True)
mangaHIall_WF50_positive_classified.write('input/mangaHIall_classified.txt', format = 'ascii.commented_header',
                                          overwrite = True)